In [1]:
%load_ext kedro.ipython

[05/16/24 14:25:27] INFO     Registered line magic '%reload_kedro'                                   ]8;id=866282;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=32930;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#51\51]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=349909;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=865377;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#53\53]8;;\

                    INFO     Resolved project path as: /Users/gavinlou/Developer/gasai.             ]8;id=741700;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=881711;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#164\164]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

24/05/16 14:25:29 WARN Utils: Your hostname, Gavins-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.241 instead (on interface en0)
24/05/16 14:25:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/16 14:25:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[05/16/24 14:25:31] INFO     Kedro project gasAI                                                    ]8;id=451586;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=152979;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#134\134]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=958993;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=635913;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#135\135]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=622050;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=798584;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#141\141]8;;\

In [ ]:
%reload_ext kedro.ipython

In [2]:
df = catalog.load("s3_conc_aligned_df")

[05/16/24 14:25:42] INFO     Loading data from s3_conc_aligned_df (ParquetDataset)...           ]8;id=786195;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=349021;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

In [15]:
data_set.head()

,exp_no,timestamp_bin,A1_Resistance,A1_Resistance_diff,A1_Resistance_norm,A1_Sensor,A1_Sensor_diff,A1_Sensor_norm,SHT40_Humidity,SHT40_temp,index,resistance_ratio,ace_conc,expo_time
0,0,0.0,720650.750,0.000,1.000000,4518.0,0.0,1.000000,42.835,29.430,4481.5,1.768473,3.033000e-07,3.0
1,0,1.0,720361.875,0.000,0.999599,4519.5,0.0,1.000332,42.840,29.435,4483.5,1.768473,3.033000e-07,3.0
2,0,2.0,719688.470,-673.405,0.998665,4523.0,3.5,1.001107,42.845,29.445,4485.5,1.768473,3.033000e-07,3.0
3,0,3.0,719015.940,-1634.810,0.997731,4526.5,8.5,1.001881,42.830,29.435,4487.5,1.768473,3.033000e-07,3.0
4,0,4.0,718727.905,-1345.095,0.997332,4528.0,7.0,1.002213,42.845,29.440,4489.5,1.768473,3.033000e-07,3.0


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import torch

# Check for missing values and handle them if necessary
df.fillna(df.mean(), inplace=True)

# Define the features and target
initial_features = ['timestamp_bin', 'A1_Resistance']
target = 'resistance_ratio'

# Extract the feature matrix and target vector
X = df[initial_features]
y = df[target]
groups = df['exp_no']

# Initialize the GroupShuffleSplit object for training/validation split
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Split into training and temporary (validation + testing) sets
train_idx, temp_idx = next(gss.split(X, y, groups=groups))

# Create DataFrames for training and temporary sets
train_df = df.iloc[train_idx]
temp_df = df.iloc[temp_idx]

# Split the temporary set into validation and testing sets
gss_temp = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
val_idx, test_idx = next(gss_temp.split(temp_df[initial_features], temp_df[target], groups=temp_df['exp_no']))

# Create DataFrames for validation and testing sets
val_df = temp_df.iloc[val_idx]
test_df = temp_df.iloc[test_idx]

# Extract features and targets for each set
X_train = train_df[initial_features].copy()
y_train = train_df[target]
X_val = val_df[initial_features].copy()
y_val = val_df[target]
X_test = test_df[initial_features].copy()
y_test = test_df[target]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Apply K-Means clustering on the training data
kmeans = KMeans(n_clusters=3, random_state=42)
X_train['cluster'] = kmeans.fit_predict(X_train_scaled)
X_train_distances = kmeans.transform(X_train_scaled)
for i in range(X_train_distances.shape[1]):
    X_train[f'distance_to_centroid_{i}'] = X_train_distances[:, i]

# Apply the same transformation to the validation and test data
X_val['cluster'] = kmeans.predict(X_val_scaled)
X_val_distances = kmeans.transform(X_val_scaled)
for i in range(X_val_distances.shape[1]):
    X_val[f'distance_to_centroid_{i}'] = X_val_distances[:, i]

X_test['cluster'] = kmeans.predict(X_test_scaled)
X_test_distances = kmeans.transform(X_test_scaled)
for i in range(X_test_distances.shape[1]):
    X_test[f'distance_to_centroid_{i}'] = X_test_distances[:, i]

# Update the features list to include the new cluster features
features = initial_features + ['cluster'] + [f'distance_to_centroid_{i}' for i in range(X_train_distances.shape[1])]

# Extract features and targets for each set again
X_train = X_train[features]
X_val = X_val[features]
X_test = X_test[features]

# Normalize the features again after adding cluster information
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

import torch
from torch.utils.data import Dataset, DataLoader, Sampler

class GroupDataset(Dataset):
    def __init__(self, X, y, groups):
        self.X = X
        self.y = y
        self.groups = groups

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.groups[idx]

class GroupBatchSampler(Sampler):
    def __init__(self, groups, batch_size):
        self.groups = groups
        self.batch_size = batch_size

        # Create a dictionary mapping group labels to indices
        self.group_to_indices = {}
        for idx, group in enumerate(groups):
            if group not in self.group_to_indices:
                self.group_to_indices[group] = []
            self.group_to_indices[group].append(idx)

        # Create a list of group batches
        self.group_batches = []
        for group, indices in self.group_to_indices.items():
            for i in range(0, len(indices), batch_size):
                self.group_batches.append(indices[i:i + batch_size])

    def __iter__(self):
        # Shuffle the group batches
        np.random.shuffle(self.group_batches)
        for batch in self.group_batches:
            yield batch

    def __len__(self):
        return len(self.group_batches)

batch_size = 64

# Create datasets
train_dataset = GroupDataset(X_train_tensor, y_train_tensor, train_df['exp_no'].values)
val_dataset = GroupDataset(X_val_tensor, y_val_tensor, val_df['exp_no'].values)
test_dataset = GroupDataset(X_test_tensor, y_test_tensor, test_df['exp_no'].values)

# Create batch samplers
train_batch_sampler = GroupBatchSampler(train_dataset.groups, batch_size)
val_batch_sampler = GroupBatchSampler(val_dataset.groups, batch_size)
test_batch_sampler = GroupBatchSampler(test_dataset.groups, batch_size)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_sampler=train_batch_sampler)
val_loader = DataLoader(val_dataset, batch_sampler=val_batch_sampler)
test_loader = DataLoader(test_dataset, batch_sampler=test_batch_sampler)


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, out_channels=16, kernel_size=3, padding=1)
        # self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32, 16)  # Adjust the input features depending on the output size after conv and pool layers
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define hyperparameters
input_channels = X_train_tensor.shape[1]
num_classes = 1
num_epochs = 30
learning_rate = 0.001

# Initialize the model, loss function, and optimizer
model = CNNModel(input_channels, num_classes)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels, _ in train_loader:
        inputs = inputs.unsqueeze(1)  # Add channel dimension
        inputs = inputs.transpose(1,2)
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels, _ in val_loader:
            inputs = inputs.unsqueeze(1)  # Add channel dimension
            inputs = inputs.transpose(1,2)
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            val_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}')

# Evaluate the model on the test set
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, labels, _ in test_loader:
        inputs = inputs.unsqueeze(1)  # Add channel dimension
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        test_loss += loss.item()

print(f'Test Loss: {test_loss/len(test_loader)}')

# Example prediction
example_input = X_test_tensor[0].unsqueeze(0).unsqueeze(1)  # Add batch and channel dimensions
example_output = model(example_input)
print(f'Predicted: {example_output.item()}, Actual: {y_test_tensor[0].item()}')


Epoch 1/30, Train Loss: 0.03349963431602749, Val Loss: 0.006057942984642927
Epoch 2/30, Train Loss: 0.005954435429896714, Val Loss: 0.005088500338595633
Epoch 3/30, Train Loss: 0.004666321922745024, Val Loss: 0.005356165309803588
Epoch 4/30, Train Loss: 0.004145314781194434, Val Loss: 0.005115919328207885
Epoch 5/30, Train Loss: 0.003781359005898869, Val Loss: 0.0033114826909671387
Epoch 6/30, Train Loss: 0.0035345771018723063, Val Loss: 0.0035386193067623467
Epoch 7/30, Train Loss: 0.0034764147034055904, Val Loss: 0.002520587650737091
Epoch 8/30, Train Loss: 0.00329661857234135, Val Loss: 0.004428987667183159
Epoch 9/30, Train Loss: 0.0032083083669978882, Val Loss: 0.0035306766581295053
Epoch 10/30, Train Loss: 0.003096125136290919, Val Loss: 0.005519613310768286
Epoch 11/30, Train Loss: 0.0029661771514993226, Val Loss: 0.002153076879948867
Epoch 12/30, Train Loss: 0.0030341977394195154, Val Loss: 0.0022319367816057223
Epoch 13/30, Train Loss: 0.0029428806761039625, Val Loss: 0.003370

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:69                                                                                   │
│                                                                                                  │
│   66 with torch.no_grad():                                                                       │
│   67 │   for inputs, labels, _ in test_loader:                                                   │
│   68 │   │   inputs = inputs.unsqueeze(1)  # Add channel dimension                               │
│ ❱ 69 │   │   outputs = model(inputs)                                                             │
│   70 │   │   loss = criterion(outputs, labels.unsqueeze(1))                                      │
│   71 │   │   test_loss += loss.item()                                                            │
│   72                                                                                             │
│                                                                                                  │
│ /Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/torch/nn/modules │
│ /module.py:1532 in _wrapped_call_impl                                                            │
│                                                                                                  │
│   1529 │   │   if self._compiled_call_impl is not None:                                          │
│   1530 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]        │
│   1531 │   │   else:                                                                             │
│ ❱ 1532 │   │   │   return self._call_impl(*args, **kwargs)                                       │
│   1533 │                                                                                         │
│   1534 │   def _call_impl(self, *args, **kwargs):                                                │
│   1535 │   │   forward_call = (self._slow_forward if torch._C._get_tracing_state() else self.fo  │
│                                                                                                  │
│ /Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/torch/nn/modules │
│ /module.py:1541 in _call_impl                                                                    │
│                                                                                                  │
│   1538 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1539 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1540 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1541 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1542 │   │                                                                                     │
│   1543 │   │   try:                                                                              │
│   1544 │   │   │   result = None                                                                 │
│                                                                                                  │
│ in forward:17                                                                                    │
│                                                                                                  │
│   14 │   │   self.fc2 = nn.Linear(16, 1)                                                         │
│   15 │                                                                                           │
│   16 │   def forward(self, x):                                                                   │
│ ❱ 17 │   │   x = F.relu(self.conv1(x))                                                           │
│   18 │   │   x = F.relu(self.conv2(x))                     